In [1]:
# Import necessary libraries for data handling (pandas) and making HTTP requests
# third-party request library to make a GET requests to the API NYT URL. An API request is just a function
# request has several functions and the one we will use is "requests.get". it creates a GET request to a URL
import pandas as pd
from requests import Session, ConnectionError, Timeout, TooManyRedirects


# Sends a GET request to the API and checks for errors.
# Returns the cryptocurrency data from the API response or an empty list if there's an error.
    
def fetch_data():
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
    # URL and parameters for the API request.
    # we need to make a dictionary whose values and keys are equal to the parameters for the data pull
    headers = {
        'Accepts': 'application/json',
        'X-CMC_PRO_API_KEY': '6319e77f-cfd7-4061-b0ea-a658aa5a7c96',
    }
    # this function will make a GET request to the Coin Market Cap URL and get the response back
    # The function, request.get, we are calling uses the URL, params and the headers and the value returned is stored in the response variable
    # the response variable contains alot of information about the request and how it happened and how long it took. The acutal data is stored in the json method. 
    
    try:
    # the data we collect is stored in the json method. We use .json to get the data back
    # the data returned will be a nested data structure which is a data structure that has other data structures inside
    # the data from the return response function will be stored in the response variable below.
    # And then we are gonna pass into the data variable
        response = Session().get(url, headers=headers, params={'start': '1', 'limit': '50', 'convert': 'USD'})
        response.raise_for_status()
        return response.json().get('data', [])
    except (ConnectionError, Timeout, TooManyRedirects) as e:
        print(f"Error fetching data: {e}")
        
        return []


def filter_data(df, start_date, end_date, min_market_cap, max_market_cap, min_price, max_price):
    df['date_added'] = pd.to_datetime(df['date_added']).dt.tz_localize(None)
    df['timestamp'] = pd.to_datetime('now')
    
    # Date range filter
    df = df[(df['date_added'] >= start_date) & (df['date_added'] <= end_date)]
    
    # Market cap filter
    df = df[(df['quote.USD.market_cap'] >= min_market_cap) & (df['quote.USD.market_cap'] <= max_market_cap)]
    
    # Price filter
    df = df[(df['quote.USD.price'] >= min_price) & (df['quote.USD.price'] <= max_price)]
    
    return df


def main():
    # Prompts the user for input values for date range, market cap range, and price range.
    # Calls fetch_data to retrieve data and converts it to a DataFrame.
    # Calls filter_data to apply the user-specified filters.
    # Displays the filtered DataFrame and saves it to a CSV file if any data matches the criteria.

    start_date = pd.to_datetime(input("Enter the start date (YYYY-MM-DD): ")).tz_localize(None)
    end_date = pd.to_datetime(input("Enter the end date (YYYY-MM-DD): ")).tz_localize(None)
    min_market_cap = float(input("Enter the minimum market cap: "))
    max_market_cap = float(input("Enter the maximum market cap: "))
    min_price = float(input("Enter the minimum price: "))
    max_price = float(input("Enter the maximum price: "))
    
    # Fetch display and process data
    data = fetch_data()
    if data:
        df = pd.json_normalize(data)  # Convert the json data structure to a DataFrame
        # Filter the DataFrame based on params
        filtered_df = filter_data(df, start_date, end_date, min_market_cap, max_market_cap, min_price, max_price) 
        
        
        if not filtered_df.empty:
            # Checks if the DataFrame filtered_df contains any data.
            # returns True if the DataFrame is empty (i.e., has no rows) and False otherwise.
            # Inverts the boolean value. So, if not filtered_df.empty is True if filtered_df has one or more rows of data (i.e., it's not empty).
            
            print("Filtered Data:") # Prints a message to the console indicating that the following output is the filtered data.
            print(filtered_df.head()) # Displays the first few rows of the filtered DataFrame filtered_df.
            # This method returns the first 5 rows of the DataFrame by default. It helps in quickly inspecting the content of the DataFrame without printing the entire dataset.
         
            filtered_df.to_csv('filtered_cryptocurrency_data.csv', index=False)  # This method writes the DataFrame to a CSV file.    # Save the filtered data to a CSV file
            # index=False: Specifies that the row index should not be included in the CSV file. 
            # By default, to_csv() includes the DataFrame index in the file. Setting index=False excludes it, making the CSV file cleaner.

        else:
            print("No data matches the criteria.")
    else:
        print("Failed to fetch data.")

if __name__ == "__main__":
    main()



Filtered Data:
      id      name symbol      slug  num_market_pairs          date_added  \
5     52       XRP    XRP       xrp              1358 2013-08-04 00:00:00   
6   3408      USDC   USDC  usd-coin             21224 2018-10-08 00:00:00   
7     74  Dogecoin   DOGE  dogecoin              1030 2013-12-15 00:00:00   
8  11419   Toncoin    TON   toncoin               479 2021-08-26 13:40:22   
9   2010   Cardano    ADA   cardano              1218 2017-10-01 00:00:00   

                                                tags    max_supply  \
5  [medium-of-exchange, enterprise-solutions, arr...  1.000000e+11   
6  [medium-of-exchange, stablecoin, asset-backed-...           NaN   
7  [mineable, pow, scrypt, medium-of-exchange, me...           NaN   
8  [pos, layer-1, ftx-bankruptcy-estate, dwf-labs...           NaN   
9  [dpos, pos, platform, research, smart-contract...  4.500000e+10   

   circulating_supply  total_supply  ...  quote.USD.market_cap_dominance  \
5        5.596087e+10  9.